In [ ]:
%pip install azure-identity
%pip install azure-search-documents

In [66]:

import dotenv
import openai
import os
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential

dotenv.load_dotenv()

# Initialize Azure search variables
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_AI_SEARCH_API_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_AI_SEARCH_INDEX")
AZURE_AI_SEARCH_RESULTS = os.getenv("AZURE_AI_SEARCH_RESULTS")

# Set up OpenAI client based on environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_MAX_RESPONSE = int(os.getenv("AZURE_OPENAI_MAX_RESPONSE"))

azure_credential = AzureKeyCredential(AZURE_SEARCH_API_KEY)

openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

In [96]:
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_INDEX, credential=azure_credential)

search_query = "I have a worker who helped me in my project and I want to pay him a token for the service he provided. how much can i give him?"
search_vector = get_embedding(search_query)

r = search_client.search(
        search_query,
        top=AZURE_AI_SEARCH_RESULTS, 
        vector_queries=[
                VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields="contentVector")],
        query_type="semantic",
        semantic_configuration_name="azureml-default")

# Initialize an empty list to accumulate all content and sources
all_content_list = []

# Iterate over the search results
for doc in r:
    content = doc["content"].replace("\n", " ")
    source = doc["filepath"]
    combined_content = f"Content: {content} Source: {source}"
    all_content_list.append(combined_content)  # Append combined content and source to the list
    
# Join the list into a single string with a space separator
all_content = " ".join(all_content_list)


SYSTEM_MESSAGE = """Assistant helps user questions about the circulars. 
    Answer only with the facts listed in the sources provided.
    If there isn't enough information provided, say you don't know.
    Each source has a name followed by a colon and the actual information including the source name for each fact you use.
    Use square brackets to reference the source, for example [info1.txt]"""

USER_MESSAGE = search_query + "\nSources:" + all_content

response = openai_client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT,
    temperature=0.5,
    max_tokens=AZURE_OPENAI_MAX_RESPONSE,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
    top_p=0.5
)

answer = response.choices[0].message.content
print(f"Answer: {answer}")

Answer: The sources provided do not specify a particular amount that you can give to a worker for the service they provided. The payment would be subject to the discretion of the Agency Head and the availability of funds [BUDGET-CIRCULAR-NO-2023-3-DECEMBER-14-2023.md]. It's also important to note that any payment should comply with existing laws, rules, and regulations.


In [82]:
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_INDEX, credential=azure_credential)

search_query = "I have a worker who helped me in my project and I want to pay him token for the service he provided as a thank you. how much can i give him?"
search_vector = get_embedding(search_query)

r = search_client.search(
        search_query,
        top=AZURE_AI_SEARCH_RESULTS, 
        vector_queries=[
                VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields="contentVector")],
        query_type="semantic",
        semantic_configuration_name="azureml-default")

# Initialize an empty list to accumulate all content and sources
all_content_list = []

# Iterate over the search results
for doc in r:
    content = doc["content"].replace("\n", " ")
    source = doc["filepath"]
    combined_content = f"Content: {content} Source: {source}"
    all_content_list.append(combined_content)  # Append combined content and source to the list
    
# Join the list into a single string with a space separator
all_content = " ".join(all_content_list)

#print all_content into a txt file
with open("all_content.txt", "w") as file:
    
    file.write(all_content)